# Introduction to the Fourier Transform using Julia

## Initialization

In [1]:
# to install the packages used in this notebook run the following
#using Pkg
#Pkg.add(["FileIO","WAV","LibSndFile","FFTW","Plots","PlotlyJS","ORCA","Interact","WebIO"])
#using WebIO
#WebIO.install_jupyter_nbextension()

In [2]:
# load needed packages
import LibSndFile
using FileIO: load, save, loadstreaming, savestreaming
using WAV, FFTW, Plots, Interact

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-15831388491411246393\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-2180311050080757330\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-15831388491411246393\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

In [3]:
# enable plotlyjs backend for interactive plots
plotlyjs()

Plots.PlotlyJSBackend()

In [4]:
# helper function
# displays an html player for a given wav file in the notebook
audioplayer(filepath) = """
    <head>
    <meta http-equiv="Content-Type" content="text/html; charset=utf-8">
    <title>Simple Test</title>
    </head>
    
    <body>
    <audio controls="controls" style="width:600px" >
      <source src="$filepath" type="audio/wav" />
      Your browser does not support the audio element.
    </audio>
    </body>
"""

audioplayer (generic function with 1 method)

# Nature of a simple wave

In [5]:
function wave(frequency,amplitude,fs,τ)
    # the semicolon (;) in Julia is not obligatory but prevents value emmision aka printing
    #= 
    frequency: cycles per second
    amplitude: multiplier to the sin
    τ:         total time duration of the wave
    fs:        samples per second
    =#
    xs = range(0, τ, length=τ*fs) .* (2pi * frequency);
    sin.(xs) .* amplitude # last value to be emmited is returned automatically
end

wave (generic function with 1 method)

![Alt Text](./Sine_curve_drawing_animation.gif)

In [11]:
@manipulate for frequency = 1:10, amplitude=0:0.1:10, fs=10:1000, τ=1:10
    xs = range(0, τ, length=τ*fs);
    plot(xs, wave(frequency,amplitude,fs,τ), size=(1300,300))
end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["frequency"], Dict{Symbol,Any}(:className => "interact ",:style => Dict{Any,Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol,Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:max => 10,:min => 1,:attributes => Dict{Any,Any}(:type => "range",Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient" => "horizontal"),:step => 1,:className => "slider slider is-fullwidth",:style => Dict{Any,Any}()))], Dict{Symbol,Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol,Any}(:className => "interact-flex-row-right"))], Dict{Symbol,Any}(:className => "interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing),"index" => (Observable{Int64} with 2 listeners. Value:
5, nothing)), Set(String[]), nothing, Asset[Asset("js", "knockout", "/home/spiros/.julia/packages/Knockout/IP1uR/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/home/spiros/.julia/packages/Knockout/IP1uR/src/../assets/knockout_punches.js"), Asset("js", nothing, "/home/spiros/.julia/packages/InteractBase/9mFwe/src/../assets/all.js"), Asset("css", nothing, "/home/spiros/.julia/packages/InteractBase/9mFwe/src/../assets/style.css"), Asset("css", nothing, "/home/spiros/.julia/packages/Interact/SbgIk/src/../assets/bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set(AbstractConnection[]), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x00007f113105ae60, Task (runnable) @0x00007f113105ae60), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"1\",\"2\",\"3\",\"4\",\"5\",\"6\",\"7\",\"8\",\"9\",\"10\"],\"changes\":WebIO.getval({\"name\":\"changes\",\"scope\":\"17005930517286626445\",\"id\":\"ob_02\",\"type\":\"observable\"}),\"index\":WebIO.getval({\"name\":\"index\",\"scope\":\

In [12]:
frequency = 2
amplitude = 5
fs = 1000
τ = 10
w = wave(frequency,amplitude,fs,τ);

In [ ]:
plot(w)

# The Fourier Transform applied to a simple wave

In [ ]:
#fft(w) .|> (x->sqrt(real(x)^2 + imag(x)^2)) |> plot

In [ ]:
#fftfreq(length(w),fs) |> plot

In [ ]:
F = fft(w) |> fftshift
freqs = fftfreq(length(w), fs) |> fftshift

time_domain = plot(range(0,τ,length=τ*fs), w, title = "Signal")
freq_domain = plot(freqs, abs.(F), title = "Spectrum", xlim=(-10, +10)) 
plot(time_domain, freq_domain, layout = 2, size=(900,400))

In [ ]:
frequency = 3
amplitude = 5
fs = 1000
τ = 10
w2 = wave(frequency,amplitude,fs,τ);

In [ ]:
w3 = w .+ w2;
plot(w3)

In [ ]:
F = fft(w3) |> fftshift
freqs = fftfreq(length(w3), fs) |> fftshift

time_domain = plot(range(0,τ,length=τ*fs), w3, title = "Signal")
freq_domain = plot(freqs, abs.(F), title = "Spectrum", xlim=(-10, +10)) 
plot(time_domain, freq_domain, layout = 2, size=(900,400))

In [ ]:
fft(w3) |> fftshift .|> abs |>  plot

# Mathematics of the Fourier Transform

In [ ]:
# add math

![](https://upload.wikimedia.org/wikipedia/commons/7/72/Fourier_transform_time_and_frequency_domains_%28small%29.gif)

# Real Life example of FFT

In [ ]:
display("text/html", audioplayer("./audio.wav"))

In [ ]:
y, fs, nbits, opt = wavread("./audio.wav");

In [ ]:
size(y), fs, nbits

196800 points, 48K metrhseis / sec

In [ ]:
nbits, typeof(nbits)

In [ ]:
# calculate duration of the audio clip
size(y)[1] / fs

4.1 deuterolepta

In [ ]:
times = collect(1:length(y)) / fs;

In [ ]:
plot(times,y)

In [ ]:
#=
yf = fft(y)

fft(y) .|> (x->sqrt(real(x)^2 + imag(x)^2)) |> plot

yif = ifft(y)

wavwrite(yif, "./ifft_audio.wav")

fft(y) .|> (x->sqrt(real(x)^2 + imag(x)^2)) |> histogram
=#